In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read data from combined dataset
data = pd.read_csv('/content/drive/My Drive/combined_data.csv')

In [ ]:
from sklearn.model_selection import train_test_split

#data.head(5)
# Sort by tiles
tile_groups = data[['Date', 'Reference']].drop_duplicates()

# Split groups — this keeps all 9 neighbors together
group_train, group_test = train_test_split(
    tile_groups, test_size=0.2, random_state=1234
)


# Join back to original data to get full rows
data_train = data.merge(group_train, on=['Date', 'Reference'], how='inner')
data_test = data.merge(group_test, on=['Date', 'Reference'], how='inner')

X = data.iloc[:, 5:10]
feature_columns = X.columns
feature_columns = list(feature_columns) + ['Date', 'Reference']

X_train = data_train[feature_columns]  # replace with your actual feature names
y_train = data_train['FROST']

X_test = data_test[feature_columns]
y_test = data_test['FROST']

#print(X_train.head(5))

# Scale X data
scaler = MinMaxScaler()
X_train[X.columns] = scaler.fit_transform(X_train[X.columns])
X_test[X.columns] = scaler.transform(X_test[X.columns])


X_train['Date'] = pd.to_datetime(X_train['Date'], format='%Y%m%d')
X_train['Date'] = X_train['Date'].apply(lambda x: x.toordinal())
X_train[X.columns] = scaler.transform(X_train[X.columns])

X_test['Date'] = pd.to_datetime(X_test['Date'], format='%Y%m%d')
X_test['Date'] = X_test['Date'].apply(lambda x: x.toordinal())
X_test[X.columns] = scaler.transform(X_test[X.columns])

X_train.head(5)

<ipython-input-5-b7a57b29ec76>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[X.columns] = scaler.fit_transform(X_train[X.columns])
<ipython-input-5-b7a57b29ec76>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[X.columns] = scaler.transform(X_test[X.columns])
<ipython-input-5-b7a57b29ec76>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,WS2M,T2MDEW,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN,T2M_MAX,Date,Reference
0,0.008347,0.515083,-0.485345,-0.009307,0.315307,730121,DC
1,0.007515,0.516224,-0.481557,-0.012458,0.315514,730122,DC
2,0.012839,0.516338,-0.481471,-0.016966,0.315917,730123,DC
3,0.006711,0.511611,-0.494757,-0.008535,0.313481,730125,DC
4,0.008374,0.511729,-0.494657,-0.009258,0.314111,730126,DC


In [ ]:
# Finding unique dates and cities to loop through
date_list = sorted(X_train['Date'].unique())
city_list = X_train['Reference'].unique()

# Remove the last few so we don't try to go into the future when it doesn't exist
date_list = date_list[:-4]
# Sample some
date_list = date_list[::2]

num_pixels = 9
sequence_length = 4

sequence_data_train = []
sequence_labels_train = []

for date in date_list:
    for city in city_list:
        # Initialize a list to hold a sequence of data
        sequence_data = []
        sequence_labels = []
        # Loop through the next sequence_length dates
        for i in range(sequence_length):
            # Get the data for the next date for all pixels in the given city
            next_date_group = X_train[(X_train['Date'] == date + i) & (X_train['Reference'] == city)]
            if len(next_date_group) == num_pixels:
                sequence_data.append(next_date_group[X.columns].values)
            else:
                # If data for next_date_group is not complete, skip to the next sequence
                break
        else:  # This else clause executes when the loop finished normally (no breaks)
            # append if compplete
            sequence_data_train.append(sequence_data)
            # Doing here to make sure the latest date is added as a label
            sequence_labels.append(y_train[next_date_group.index].values)
            # If there is any frost in the city, we consider it frost
            sequence_labels_train.append(max(y_train[next_date_group.index].values))

num_features = 5
output_dim = 1

# lists to numpy arrays
sequence_data_train = np.array(sequence_data_train)
sequence_labels_train = np.array(sequence_labels_train)

# Reshape data to (num_samples, sequence_length, num_pixels, num_features)
sequence_data_train = sequence_data_train.reshape(-1, sequence_length, num_pixels, num_features)
# Reshape labels to (num_samples, output_dim)
sequence_labels_train = sequence_labels_train.reshape(-1, output_dim)

sequence_data_train = torch.tensor(sequence_data_train, dtype=torch.float32)
sequence_labels_train = torch.tensor(sequence_labels_train, dtype=torch.float32)

print(sequence_data_train.shape)  # Check shapes
print(sequence_labels_train.shape)

train_dataset = TensorDataset(sequence_data_train, sequence_labels_train)

# small batch size for better training stability
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


torch.Size([13809, 4, 9, 5])
torch.Size([13809, 1])


In [ ]:
# small batch size for better training stability
batch_size = 32

# Create a DataLoader from the TensorDataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
date_list_test = sorted(X_test['Date'].unique())
city_list_test = X_test['Reference'].unique()

# Remove the last few so we don't try to go into the future when it doesn't exist
date_list_test = date_list_test[:-4]

sequence_data_test = []
sequence_labels_test = []

for date in date_list_test:
    for city in city_list_test:
        # Initialize a list to hold a sequence of data
        sequence_data = []
        sequence_labels = []
        # Loop through the next sequence_length dates
        for i in range(sequence_length):
            # Get the data for the next date for all pixels in the given city
            next_date_group = X_test[(X_test['Date'] == date + i) & (X_test['Reference'] == city)]
            if len(next_date_group) == num_pixels:
                sequence_data.append(next_date_group[X.columns].values)
            else:
                # If data for next_date_group is not complete, skip to the next sequence
                break
        else:  # This else clause executes when the loop finished normally (no breaks)
            sequence_data_test.append(sequence_data)
            # Doing here to make sure the latest date is added as a label
            sequence_labels.append(y_test[next_date_group.index].values)
            # If there is any frost in the city, we consider it frost
            sequence_labels_test.append(max(y_test[next_date_group.index].values))

sequence_data_test = np.array(sequence_data_test)
sequence_labels_test = np.array(sequence_labels_test)

# Reshape data to (num_samples, sequence_length, num_pixels, num_features)
sequence_data_test = sequence_data_test.reshape(-1, sequence_length, num_pixels, num_features)
# Reshape labels to (num_samples, output_dim)
sequence_labels_test = sequence_labels_test.reshape(-1, output_dim)

# Convert data and labels to PyTorch tensors
sequence_data_test = torch.tensor(sequence_data_test, dtype=torch.float32)
sequence_labels_test = torch.tensor(sequence_labels_test, dtype=torch.float32)

print(sequence_data_test.shape)  # Check shapes
print(sequence_labels_test.shape)

test_dataset = TensorDataset(sequence_data_test, sequence_labels_test)

# Use a batch size for testing (can be larger since we don't update gradients)
test_batch_size = 64

test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

torch.Size([102, 4, 9, 5])
torch.Size([102, 1])


In [ ]:
import pickle
'''
f = open('/content/drive/My Drive/train_loader.pkl', 'wb')
pickle.dump(train_loader, f)
f.close()

'''
f = open('/content/drive/My Drive/train_loader.pkl', 'rb')
train_loader = pickle.load(f)
f.close()


In [ ]:
# Feature size
dynamic_input_dim = X.shape[1]

# Define the model
class TemporalNet(nn.Module):
    def __init__(self, num_features=5, num_pixels=9, hidden_dim=32, output_dim=1):
        super(TemporalNet, self).__init__()
        self.convs = nn.ModuleList([nn.Conv1d(in_channels=num_pixels, out_channels=1, kernel_size=1) for _ in range(num_features)])
        self.lstm = nn.LSTM(num_features, hidden_dim, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.split(x, 1, dim=3)
        x = [F.relu(conv(feature.squeeze(3).permute(0, 2, 1))) for feature, conv in zip(x, self.convs)]
        x = torch.cat(x, dim=1)
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

# Initialize model and parameters
hidden_dim = 64
output_dim = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TemporalNet(num_features=num_features, num_pixels=num_pixels, hidden_dim=hidden_dim, output_dim=output_dim).to(device)

# Loss and optimizer
pos_weight = torch.ones([1]) * 5
pos_weight = pos_weight.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Reduced learning rate

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training Loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        y_batch = y_batch.float().view(-1, 1)
        loss = criterion(outputs, y_batch)
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()

    # Step the scheduler
    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')



Epoch 1/25, Loss: 1.0074
Epoch 2/25, Loss: 0.8088
Epoch 3/25, Loss: 0.7911
Epoch 4/25, Loss: 0.7448
Epoch 5/25, Loss: 0.7480
Epoch 6/25, Loss: 0.5982
Epoch 7/25, Loss: 0.5812
Epoch 8/25, Loss: 0.5981
Epoch 9/25, Loss: 0.5765
Epoch 10/25, Loss: 0.5816
Epoch 11/25, Loss: 0.5650
Epoch 12/25, Loss: 0.5673
Epoch 13/25, Loss: 0.5651
Epoch 14/25, Loss: 0.5661
Epoch 15/25, Loss: 0.5687
Epoch 16/25, Loss: 0.5687
Epoch 17/25, Loss: 0.5658
Epoch 18/25, Loss: 0.5648
Epoch 19/25, Loss: 0.5640
Epoch 20/25, Loss: 0.5642
Epoch 21/25, Loss: 0.5659
Epoch 22/25, Loss: 0.5640
Epoch 23/25, Loss: 0.5646
Epoch 24/25, Loss: 0.5650
Epoch 25/25, Loss: 0.5638


In [ ]:
# Evaluation
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits)
        y_pred.append(probs.cpu().numpy())
        y_batch = y_batch.float()
        y_batch = y_batch.view(-1, 1)
        y_true.append(y_batch.cpu().numpy())

y_pred = np.vstack(y_pred)
y_true = np.vstack(y_true)

In [ ]:
print(y_pred.shape)
print(y_true.shape)

(102, 1)
(102, 1)


Now we have predictions in the form of probablities for each pixel. Let's convert these into binary predictions. Note that our NN was tuned with a loss function called BCEWithLogitsLoss, which combines binary classification error and log-loss for predicted values compared to ground truth.

In [ ]:
predictions = []
probabilities = []
for prediction in y_pred:
    if prediction > 0.8:
        predictions.append(1)
    else:
        predictions.append(0)
    probabilities.append(prediction)

In [ ]:
f = open('/content/drive/My Drive/predictions.pkl', 'wb')
pickle.dump(predictions, f)
f.close()

f = open('/content/drive/My Drive/y_true_list.pkl', 'wb')
pickle.dump(list(y_true), f)
f.close()

In [3]:
import pickle

f = open('/content/drive/My Drive/y_true_list.pkl', 'rb')
y_true = pickle.load(f)
f.close()

f = open('/content/drive/My Drive/predictions.pkl', 'rb')
predictions = pickle.load(f)
f.close()

In [ ]:
from sklearn import metrics

accuracy = metrics.accuracy_score(list(y_true), predictions)
print(f'accuracy: {accuracy}')

precision = metrics.precision_score(list(y_true), predictions)
print(f'precision: {precision}')

recall = metrics.recall_score(list(y_true), predictions)
print(f'recall: {recall}')

f1 = metrics.f1_score(list(y_true), predictions)
print(f'f1 score: {f1}')

accuracy: 0.8823529411764706
precision: 0.6666666666666666
recall: 0.7368421052631579
f1 score: 0.7


In [8]:
from sklearn import metrics
auc = metrics.roc_auc_score(list(y_true), predictions)
print(f'auc score: {auc}')

tn, fp, fn, tp = metrics.confusion_matrix(list(y_true), predictions).ravel()
score = -2*fn-fp+tn
print(f'score: {score}')

print(metrics.confusion_matrix(list(y_true), predictions))

auc score: 0.8262523779327838
score: 59
[[76  7]
 [ 5 14]]
